**NOMBRE: QUISPE SUCULLANI JOSE DAVID**
**CARRERA: ING CIENCIAS DE LA COMPUTACAION**
**GITHUB:**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ===================================================================
# PASO 1: INSTALAR LIBRERÍAS DE ÚLTIMA GENERACIÓN
# ===================================================================
!pip install transformers torch sentencepiece Pillow pandas -q

import os
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import csv
from PIL import Image
import re
import torch
from transformers import DonutProcessor, VisionEncoderDecoderModel
from IPython.display import display

In [ ]:

# Configurar dispositivo (muy recomendable usar GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Dispositivo configurado: {device}")
print(f"✅ Google Drive montado correctamente.")

In [ ]:
# ===================================================================
# CONFIGURACIÓN CENTRALIZADA - ¡AJUSTA ESTAS RUTAS!
# ===================================================================
CONFIG = {
    # 1. Ruta a tu CSV con las coordenadas. AHORA SOLO SE USA PARA OBTENER LA LISTA DE PREGUNTAS.
    "coordenadas_csv_path": "/content/drive/MyDrive/Inteligencia-Artificial_2/Proyecto-20250829T014112Z-1-001/Proyecto/MapaCoordenadas/Coordenadas.csv",

    # 2. Ruta a la CARPETA con TODAS las imágenes que quieres procesar.
    "carpeta_a_procesar": "/content/drive/MyDrive/Inteligencia-Artificial_2/Argentina-20250829T015557Z-1-001/Argentina",

    # 3. Ruta al CSV donde se guardarán los resultados finales.
    "output_csv_path": "/content/drive/MyDrive/resultados_finales_con_donut.csv",
}


In [ ]:
# ===================================================================
# 1. GESTIÓN DEL ARCHIVO CSV DE SALIDA
# ===================================================================
CSV_HEADERS = []

def crear_csv_salida(headers):
    global CSV_HEADERS
    CSV_HEADERS = headers
    with open(CONFIG["output_csv_path"], mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(CSV_HEADERS)
    print(f"✅ Archivo CSV de resultados creado/limpiado en: {CONFIG['output_csv_path']}")

def guardar_en_csv(datos_acta):
    with open(CONFIG["output_csv_path"], mode='a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=CSV_HEADERS, extrasaction='ignore')
        writer.writerow(datos_acta)


In [ ]:

# ===================================================================
# 2. PROCESADOR DE ACTAS (CON MOTOR DONUT)
# ===================================================================
class ProcesadorActasDonut:
    def __init__(self, campos_a_extraer):
        print("Iniciando motor Donut... (Esto puede tardar varios minutos la primera vez)")
        # Cargar el procesador y el modelo pre-entrenado de Hugging Face para Preguntas y Respuestas en Documentos
        self.processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")
        self.model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa").to(device)
        self.campos = campos_a_extraer
        print(f"✅ Modelo Donut listo. Se harán {len(self.campos)} preguntas por acta.")

    def limpiar_respuesta(self, texto_crudo, campo):
        """Limpia la respuesta del modelo Donut."""
        if texto_crudo is None: return None
        # Lógica de limpieza inteligente
        tipo_de_dato = 'numero' if any(k in campo.upper() for k in ['VOTOS', 'ELECTORES', 'PAPELETAS', 'MESA']) else 'texto'
        if tipo_de_dato == 'numero':
            numeros = ''.join(filter(str.isdigit, texto_crudo))
            try: return int(numeros) if numeros else None
            except ValueError: return None
        else:
            return " ".join(texto_crudo.split()).upper()

    def procesar_acta_completa(self, imagen_path):
        """Abre una imagen y le hace preguntas al modelo Donut para cada campo."""
        try:
            imagen = Image.open(imagen_path).convert("RGB")
        except Exception as e:
            print(f"❌ No se pudo abrir la imagen: {imagen_path}, error: {e}")
            return None

        image_filename = os.path.basename(imagen_path)
        departamento = os.path.basename(os.path.dirname(imagen_path))
        resultados = {'departamento': departamento, 'imagen_path': imagen_path}
        print(f"\n--- Interrogando al acta: {image_filename} ---")

        for campo in self.campos:
            # Crear la "pregunta" para el modelo
            task_prompt = f"<s_docvqa><s_question>{campo}</s_question><s_answer>"
            decoder_input_ids = self.processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids.to(device)

            # Procesar la imagen
            pixel_values = self.processor(imagen, return_tensors="pt").pixel_values.to(device)

            # Generar la respuesta
            outputs = self.model.generate(
                pixel_values,
                decoder_input_ids=decoder_input_ids,
                max_length=self.model.decoder.config.max_position_embeddings,
                pad_token_id=self.processor.tokenizer.pad_token_id,
                eos_token_id=self.processor.tokenizer.eos_token_id,
                use_cache=True,
                bad_words_ids=[[self.processor.tokenizer.unk_token_id]],
                return_dict_in_generate=True,
            )

            # Decodificar y limpiar la respuesta
            respuesta_cruda = self.processor.batch_decode(outputs.sequences, skip_special_tokens=True)[0]
            # Extraer solo el contenido de la respuesta
            respuesta_contenido = re.sub(r"<s_question>.*</s_question><s_answer>", "", respuesta_cruda).strip()

            valor_limpio = self.limpiar_respuesta(respuesta_contenido, campo)
            resultados[campo] = valor_limpio
            print(f"   ❔ Pregunta: '{campo}' -> Respuesta: '{valor_limpio}'")

        guardar_en_csv(resultados)

In [ ]:


# ===================================================================
# 3. FUNCIÓN PRINCIPAL DE PROCESAMIENTO
# ===================================================================
def run_processing():
    print("\n" + "="*60); print("✨ INICIANDO PROCESO CON IA DE COMPRENSIÓN DE DOCUMENTOS (DONUT)"); print("="*60)
    try:
        coordenadas_df = pd.read_csv(CONFIG['coordenadas_csv_path'])
        coordenadas_df.columns = coordenadas_df.columns.str.strip()
        # AHORA SOLO LO USAMOS PARA SACAR LA LISTA DE PREGUNTAS
        campos_a_preguntar = sorted(coordenadas_df['label_name'].unique().tolist())
        print(f"✅ Lista de {len(campos_a_preguntar)} preguntas cargada desde: {CONFIG['coordenadas_csv_path']}")
    except FileNotFoundError:
        print(f"❌ ERROR: No se encontró tu archivo de coordenadas en '{CONFIG['coordenadas_csv_path']}'."); return

    carpeta_objetivo = CONFIG["carpeta_a_procesar"]
    if not os.path.isdir(carpeta_objetivo): print(f"❌ ERROR: La carpeta a procesar '{carpeta_objetivo}' no existe."); return

    actas_a_procesar = [os.path.join(dirpath, f) for dirpath, _, fnames in os.walk(carpeta_objetivo) for f in fnames if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    if not actas_a_procesar: print(f"❌ No se encontró NINGUNA imagen en la carpeta '{carpeta_objetivo}'."); return
    print(f"✅ Se encontraron {len(actas_a_procesar)} actas para procesar.")

    headers = ['imagen_path', 'departamento', 'fecha_procesamiento'] + campos_a_preguntar
    crear_csv_salida(headers)

    procesador = ProcesadorActasDonut(campos_a_preguntar)
    for path in tqdm(actas_a_procesar, desc="Procesando Actas con Donut"):
        procesador.procesar_acta_completa(path)

    print("\n🎉 PROCESAMIENTO COMPLETADO!")
    print("\n" + "="*60); print(f"📊 ANÁLISIS DE RESULTADOS EN '{CONFIG['output_csv_path']}':"); print("="*60)
    try:
        df_resultados = pd.read_csv(CONFIG['output_csv_path'])
        print("--- Información General y Valores Nulos ---"); df_resultados.info()
        print("\n--- Vista Previa de la Tabla de Resultados ---"); display(df_resultados)
    except Exception as e: print(f"No se pudo leer o analizar el archivo CSV de resultados. Error: {e}")

# ===================================================================
# 4. EJECUTAR TODO
# ===================================================================
run_processing()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Dispositivo configurado: cuda
✅ Google Drive montado correctamente.

✨ INICIANDO PROCESO CON IA DE COMPRENSIÓN DE DOCUMENTOS (DONUT)
✅ Lista de 17 preguntas cargada desde: /content/drive/MyDrive/Inteligencia-Artificial_2/Proyecto-20250829T014112Z-1-001/Proyecto/MapaCoordenadas/Coordenadas.csv
✅ Se encontraron 545 actas para procesar.
✅ Archivo CSV de resultados creado/limpiado en: /content/drive/MyDrive/resultados_finales_con_donut.csv
Iniciando motor Donut... (Esto puede tardar varios minutos la primera vez)


preprocessor_config.json:   0%|          | 0.00/359 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.30M [00:00<?, ?B/s]

...

   ❔ Pregunta: 'VOTOS VÁLIDOS' -> Respuesta: '2'

--- Interrogando al acta: mesa_2_105194.jpg ---
   ❔ Pregunta: 'AP' -> Respuesta: 'AP 017'
   ❔ Pregunta: 'APB SÚMATE' -> Respuesta: 'APB SÚMATE 004'
   ❔ Pregunta: 'CODIGO MESA' -> Respuesta: 'None'
   ❔ Pregunta: 'ELECTORES HABILITADOS' -> Respuesta: '7'
   ❔ Pregunta: 'FP' -> Respuesta: 'FP 005'
   ❔ Pregunta: 'LIBRE' -> Respuesta: 'LIBRE 004'
   ❔ Pregunta: 'LYP ADN' -> Respuesta: 'LYP ADN APB SOMATE'
   ❔ Pregunta: 'MAS-IPSP' -> Respuesta: 'MAS-IPSP 00 1'
   ❔ Pregunta: 'MESA' -> Respuesta: '628'
   ❔ Pregunta: 'MORENA' -> Respuesta: 'MORENA 7 8 3'
   ❔ Pregunta: 'PAPELETAS EN ÁNFORA' -> Respuesta: '732'
   ❔ Pregunta: 'PAPELETAS NO UTILIZADAS' -> Respuesta: '82'
   ❔ Pregunta: 'PDC' -> Respuesta: 'PDC 022'
   ❔ Pregunta: 'UNIDAD' -> Respuesta: 'UNIDAD 001'
   ❔ Pregunta: 'VOTOS BLANCOS' -> Respuesta: '5'
   ❔ Pregunta: 'VOTOS NULOS' -> Respuesta: '82'



Procesando Actas con Donut:  92%|█████████▏| 499/545 [3:42:41<20:31, 26.76s/it]

   ❔ Pregunta: 'VOTOS VÁLIDOS' -> Respuesta: '5'

--- Interrogando al acta: mesa_4_105214.jpg ---
   ❔ Pregunta: 'AP' -> Respuesta: 'AP 0'
   ❔ Pregunta: 'APB SÚMATE' -> Respuesta: 'APB SÚMATE 0.27'
   ❔ Pregunta: 'CODIGO MESA' -> Respuesta: '4'
   ❔ Pregunta: 'ELECTORES HABILITADOS' -> Respuesta: '3'
   ❔ Pregunta: 'FP' -> Respuesta: 'FP 001'
   ❔ Pregunta: 'LIBRE' -> Respuesta: 'LIBRE 008'
   ❔ Pregunta: 'LYP ADN' -> Respuesta: 'LYP ADN 080'
   ❔ Pregunta: 'MAS-IPSP' -> Respuesta: 'MAS-IPSP 001'
   ❔ Pregunta: 'MESA' -> Respuesta: '4'
   ❔ Pregunta: 'MORENA' -> Respuesta: 'MORENA 73.00'
   ❔ Pregunta: 'PAPELETAS EN ÁNFORA' -> Respuesta: '75'
   ❔ Pregunta: 'PAPELETAS NO UTILIZADAS' -> Respuesta: '75'
   ❔ Pregunta: 'PDC' -> Respuesta: 'PDC 016'
   ❔ Pregunta: 'UNIDAD' -> Respuesta: 'UNIDAD 006'
   ❔ Pregunta: 'VOTOS BLANCOS' -> Respuesta: '3'
   ❔ Pregunta: 'VOTOS NULOS' -> Respuesta: '75'



Procesando Actas con Donut:  92%|█████████▏| 500/545 [3:43:08<20:09, 26.88s/it]

   ❔ Pregunta: 'VOTOS VÁLIDOS' -> Respuesta: '60'

--- Interrogando al acta: mesa_6_105234.jpg ---
